In [66]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-473ez7kw/gym-walk_5fc61df713f940b987ef995de9b2d304
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-473ez7kw/gym-walk_5fc61df713f940b987ef995de9b2d304
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done


In [67]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)



In [68]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [69]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")


In [70]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [71]:

def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [72]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)

In [73]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [74]:
init_state

0

In [65]:
state, reward, done, info = env.step(RIGHT)
print("state:{0} - reward:{1} - done:{2} - info:{3}".format(state, reward, done, info))

state:5 - reward:0 - done:True - info:{'prob': 1.0, 'TimeLimit.truncated': False}


In [75]:
pi_frozenlake1 = lambda s: {
    0: RIGHT,
    1: RIGHT,
    2: RIGHT,
    3: RIGHT,
    4: RIGHT,
    5: RIGHT,
    6: RIGHT,
    7: RIGHT,
    8: RIGHT,
    9: RIGHT,
    10:RIGHT,
    11:RIGHT,
    12:RIGHT,
    13:RIGHT,
    14:RIGHT,
    15:RIGHT #Stop
}[s]
print("Name:LOKESH R     ")
print("Register Number:212222240055        ")
print_policy(pi_frozenlake1, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Name:LOKESH R     
Register Number:212222240055        
Policy:
| 00      > | 01      > | 02      > | 03      > |
| 04      > |           | 06      > |           |
| 08      > | 09      > | 10      > |           |
|           | 13      > | 14      > |           |


In [77]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_frozenlake1, goal_state=goal_state)*100,
    mean_return(env, pi_frozenlake1)))


Reaches goal 1.00%. Obtains an average undiscounted return of 0.0100.


In [78]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    prev_V = np.zeros(len(P), dtype=np.float64)
    while True:
        V = np.zeros(len(P), dtype=np.float64)
        for s in range(len(P)):
            for prob, next_state, reward, done in P[s][pi(s)]:
                V[s] += prob * (reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < theta:
            break
        prev_V = V.copy()
    return V

In [102]:
V1 = policy_evaluation(pi_frozenlake1, P)
print("Name:LOKESH R")
print("Register Number:212222240055")
print_state_value_function(V1, P, n_cols=4, prec=4)


Name:LOKESH R
Register Number:212222240055
State-value function:
| 00 0.0315 | 01 0.0238 | 02 0.0476 | 03    0.0 |
| 04 0.0392 |           | 06 0.0952 |           |
| 08 0.0861 | 09  0.219 | 10 0.2381 |           |
|           | 13  0.419 | 14  0.619 |           |


In [80]:
def policy_improvement(V, P, gamma=1.0):
    Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
    for s in range(len(P)):
      for a in range(len(P[s])):
        for prob, next_state,reward, done in P[s][a]:
          Q[s][a]+= prob*(reward+gamma*V[next_state]*(not done))
          new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]

    return new_pi

In [81]:
pi_2 = policy_improvement(V1, P)
print("Name: LOKESH R ")
print("Register Number:212222240055         ")
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)


Name: LOKESH R 
Register Number:212222240055         
Policy:
| 00      < | 01      ^ | 02      < | 03      < |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [82]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state)*100,
    mean_return(env, pi_2)))

Reaches goal 66.00%. Obtains an average undiscounted return of 0.6600.


In [98]:
V2 = policy_evaluation(pi_2, P)
print("Name:      ")
print("Register Number:         ")
print_state_value_function(V2, P, n_cols=4, prec=4)

Name:      
Register Number:         
State-value function:
| 00 0.7805 | 01 0.6585 | 02 0.5366 | 03 0.2683 |
| 04 0.7805 |           | 06 0.4146 |           |
| 08 0.7805 | 09 0.7805 | 10 0.7073 |           |
|           | 13 0.8537 | 14 0.9268 |           |
Name:      
Register Number:         
State-value function:
| 00 0.7805 | 01 0.6585 | 02 0.5366 | 03 0.2683 |
| 04 0.7805 |           | 06 0.4146 |           |
| 08 0.7805 | 09 0.7805 | 10 0.7073 |           |
|           | 13 0.8537 | 14 0.9268 |           |


In [84]:
if(np.sum(V1>=V2)==16):
  print("The Adversarial policy is the better policy")
elif(np.sum(V2>=V1)==16):
  print("The Improved policy is the better policy")
else:
  print("Both policies have their merits.")

The Improved policy is the better policy


In [90]:
def policy_iteration(P, gamma=1.0,theta=1e-10):
  random_actions=np.random.choice(tuple(P[0].keys()),len(P))
  pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]
  while True:
    old_pi = {s:pi(s) for s in range(len(P))}
    V = policy_evaluation(pi, P,gamma,theta)
    pi = policy_improvement(V,P,gamma)
    if old_pi == {s:pi(s) for s in range(len(P))}:
      break
  return V,pi

In [94]:
print("Name: LOKESH R")
print("Register Number: 212222240055")
print('Optimal policy and state-value function (PI):')
print_policy(optimal_pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Name: LOKESH R
Register Number: 212222240055
Optimal policy and state-value function (PI):
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [95]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=goal_state)*100,
    mean_return(env, optimal_pi)))

Reaches goal 69.00%. Obtains an average undiscounted return of 0.6900.


In [99]:
print("Name:LOKESH R")
print("Register Number:212222240055")
print_state_value_function(optimal_V, P, n_cols=4, prec=4)


Name:LOKESH R
Register Number:212222240055
State-value function:
| 00 0.8235 | 01 0.8235 | 02 0.8235 | 03 0.8235 |
| 04 0.8235 |           | 06 0.5294 |           |
| 08 0.8235 | 09 0.8235 | 10 0.7647 |           |
|           | 13 0.8824 | 14 0.9412 |           |
